In [1]:
import multiprocessing
from langchain_community.chat_models import ChatLlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

# RAG 체인 구성
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA

from dotenv import dotenv_values

In [2]:
config = dotenv_values(
    dotenv_path = "../.env"
)

In [3]:
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

In [4]:
# RAG 시스템을 위한 LLM 설정 (KV 캐시 최적화 포함)
llm = ChatLlamaCpp(
    temperature=0.1,  # RAG에서는 더 일관된 답변을 위해 낮은 temperature 사용
    model_path=config["EXAONE-3.5-2.4B-Instruct-Q4_K_M.gguf"],
    n_ctx=16384,  # 컨텍스트 크기를 적절히 조정 (32768은 메모리 부담이 큼)
    n_gpu_layers=8,
    n_batch=32,  # KV 캐시 최적화를 위해 더 작은 배치 크기
    max_tokens=512,  # 더 긴 답변을 위해 토큰 수 증가
    n_threads=multiprocessing.cpu_count() - 1,
    repeat_penalty=1.1,  # RAG에서는 적당한 반복 패널티
    top_p=0.9,  # RAG에서는 더 높은 top_p로 다양한 답변 생성
    verbose=False,  # RAG에서는 verbose 비활성화로 출력 정리
    callback_manager=callback_manager,
    
    # KV 캐시 최적화를 위한 설정들
    use_mlock=True,  # 메모리 잠금으로 성능 향상
    use_mmap=True,   # 메모리 맵핑 사용
    
    # 추가적인 KV 캐시 관련 설정들
    # n_keep=-1,     # 모든 토큰을 캐시에 유지 (선택사항)
    # rope_scaling_type=1,  # RoPE 스케일링 (긴 컨텍스트용)
    # rope_freq_base=10000,  # RoPE 주파수 기본값
)

llama_context: n_batch is less than GGML_KQ_MASK_PAD - increasing to 64
llama_context: n_ctx_per_seq (16384) < n_ctx_train (32768) -- the full capacity of the model will not be utilized
llama_kv_cache_unified: LLAMA_SET_ROWS=0, using old ggml_cpy() method for backwards compatibility


In [5]:
# 웹사이트 내용을 읽어서 답변하는 RAG 파이프라인 예시
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA

# 웹사이트에서 문서 로드
url = "https://n.news.naver.com/article/662/0000072581?cds=news_media_pc"
loader = WebBaseLoader(url)
docs = loader.load()

# 문서 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
splits = text_splitter.split_documents(docs)

# 임베딩 및 벡터스토어 생성
# embeddings = HuggingFaceEmbeddings(model_name="jhgan/ko-sroberta-multitask")
embeddings = HuggingFaceEmbeddings(
    model_name=config["multilingual-e5-small-ko"],
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)

vectorstore = FAISS.from_documents(splits, embeddings)

# Retriever 생성
retriever = vectorstore.as_retriever()

# LLM 준비 (아래 llm은 기존 코드에서 정의된 ChatLlamaCpp 인스턴스 사용)
# llm = ChatLlamaCpp(...)

# RAG QA 체인 생성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

# 질문 예시
query = "이 뉴스의 주요 내용이 뭐야?"
result = qa_chain({"query": query})

print("답변:", result["result"])

USER_AGENT environment variable not set, consider setting it to identify your requests.
C:\Users\jw160\AppData\Local\Temp\ipykernel_21336\3102760428.py:42: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": query})


제시된 뉴스 내용은 특정 날짜와 관련된 여러 주제를 다루고 있지만, 가장 핵심적인 내용을 요약하자면:

1. **폭염과 집중호우**: 지속적인 폭염과 최근 집중호우와 같은 자연 재해에 대한 언급이 있습니다. 특히 정부의 대응 방안이나 주의사항이 포함될 가능성이 큽니다.

2. **건강 관련 이슈**: 당뇨병과 관련된 경고 신호나, 특정 질병(예: 암) 예방 및 진단에 대한 정보가 포함되어 있을 것으로 추정됩니다.

이러한 요소들이 복합적으로 작용하여 독자들에게 중요한 정보를 전달하고 있습니다.답변: 제시된 뉴스 내용은 특정 날짜와 관련된 여러 주제를 다루고 있지만, 가장 핵심적인 내용을 요약하자면:

1. **폭염과 집중호우**: 지속적인 폭염과 최근 집중호우와 같은 자연 재해에 대한 언급이 있습니다. 특히 정부의 대응 방안이나 주의사항이 포함될 가능성이 큽니다.

2. **건강 관련 이슈**: 당뇨병과 관련된 경고 신호나, 특정 질병(예: 암) 예방 및 진단에 대한 정보가 포함되어 있을 것으로 추정됩니다.

이러한 요소들이 복합적으로 작용하여 독자들에게 중요한 정보를 전달하고 있습니다.
